In [15]:
import subprocess
from flask import Flask, request, jsonify, render_template, send_from_directory
from flask_socketio import SocketIO, emit
from ultralytics import YOLO
import os, cv2
import numpy as np
from sklearn.cluster import HDBSCAN
from collections import Counter
from math import sqrt
from sklearn.linear_model import LinearRegression
import mediapipe as mp
from numpy import dot
from numpy.linalg import norm
from IsTriangle import is_triangle_line_in_box
from Average import average
from cos import cosx

MODEL_PATH = "/Users/kwonsebin/Documents/khu/4-1/capstone/runs/detect/train13/weights/best.pt"
MODEL_PATH1 = "/Users/kwonsebin/Documents/khu/4-1/capstone/runs/detect/train82/weights/best.pt"

detection_model = YOLO(MODEL_PATH)
detection_model1 = YOLO(MODEL_PATH1)

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

# 저장 폴더 생성
output_folder = "uploads"
os.makedirs(output_folder, exist_ok=True)
video_path = "/Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4"

detection_Ball_Glove = detection_model.track(video_path, save=False)
detection_results3 = detection_model1.track(video_path, save=False)


ball, glove = [], []
for i, result in enumerate(detection_Ball_Glove):
    frame_number = i + 1
    total_frames = len(detection_Ball_Glove)
    # message = f"Processing frame {frame_number}/{total_frames}"
    # print(message)  # 서버 로그 출력
    # socketio.emit('progress', {'message': message})  # WebSocket으로 전송
    
    for obj in result.boxes:
        if obj.cls == 2.0:  # 공
            x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
            ball.append([x1, y1])
        if obj.cls == 8.0:  # 글러브
            x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
            glove.append([x1, y1])

# HDBSCAN 클러스터링
# hdbBall = HDBSCAN(min_cluster_size=2, min_samples=1, cluster_selection_epsilon=10)
# hdbBall.fit(ball)
# hdbGlove = HDBSCAN(min_cluster_size=2, min_samples=1, cluster_selection_epsilon=10)
# hdbGlove.fit(glove)

# most_common_ball = Counter(hdbBall.labels_).most_common(1)[0][0]
# most_common_glove = Counter(hdbGlove.labels_).most_common(1)[0][0]

# 각 프레임 정보 저장
frame = {}
for i, result in enumerate(detection_Ball_Glove):
    info = {}
    for obj in result.boxes:
        if obj.cls == 2.0:  # 공
            x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
            # if hdbBall.labels_[ball.index([x1, y1])] == most_common_ball:
                # info["ball"] = (x1, y1, x2, y2)
            info["ball"] = (x1, y1, x2, y2)
        if obj.cls == 8.0:  # 글러브
            x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())
            # if hdbGlove.labels_[glove.index([x1, y1])] == most_common_glove:
                # info["glove"] = (x1, y1, x2, y2)
            info["glove"] = (x1, y1, x2, y2)
    frame[i] = info

# 거리 계산
distances = {}
for frame_number, objects in frame.items():
    if "ball" in objects and "glove" in objects:
        ball_coords = objects["ball"]
        glove_coords = objects["glove"]
        ball_center = ((ball_coords[0] + ball_coords[2]) / 2, (ball_coords[1] + ball_coords[3]) / 2)
        glove_center = ((glove_coords[0] + glove_coords[2]) / 2, (glove_coords[1] + glove_coords[3]) / 2)
        distance = sqrt((ball_center[0] - glove_center[0]) ** 2 + (ball_center[1] - glove_center[1]) ** 2)
        distances[frame_number] = distance

# 선형 회귀
X = np.array(list(distances.keys())).reshape(-1, 1)[:-1]
y = np.array(list(distances.values()))[:-1]
model = LinearRegression()
model.fit(X, y)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



I0000 00:00:1734524969.839355 13610192 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


video 1/1 (frame 1/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 131.9ms


W0000 00:00:1734524970.107996 20498563 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


video 1/1 (frame 2/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 119.6ms
video 1/1 (frame 3/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 91.9ms


W0000 00:00:1734524970.380034 20498563 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


video 1/1 (frame 4/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 1 Glove, 46.7ms
video 1/1 (frame 5/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 1 Glove, 113.8ms
video 1/1 (frame 6/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 1 Glove, 56.4ms
video 1/1 (frame 7/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 1 Glove, 34.9ms
video 1/1 (frame 8/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 1 Glove, 56.9ms
video 1/1 (frame 9/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 1 Glove, 51.5ms
video 1/1 (frame 10/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_ball, 1 Glove, 40.1ms
video 1/1 (frame 11/106) /Users/kwonsebin/Documents/khu/4-1/capstone/new/video/19.mp4: 384x640 1 Baseball_bal

LinearRegression()

In [16]:

print(len(X), len(y))

for i in range(len(X)):
    print(X[i], y[i])

24 24
[3] 306.47104267777075
[4] 287.8632487831679
[5] 274.22709567072326
[6] 263.7650469641495
[7] 244.8999795835026
[8] 230.92477130009246
[9] 219.69296757065302
[10] 204.0006127441778
[11] 191.47911113225902
[12] 181.1884378209603
[13] 168.84090144274876
[14] 158.03876106828983
[15] 145.9469081549863
[16] 135.15176654413364
[17] 124.17930584441193
[18] 115.59628021696892
[19] 104.52870419171951
[20] 94.50132274206535
[21] 85.44003745317531
[22] 81.56745674593515
[23] 75.31600095597217
[24] 65.3490627323759
[25] 60.4648658313239
[26] 51.92783453986888


In [12]:
model

LinearRegression()

In [19]:
answer = int(-model.intercept_ / model.coef_[0]) + 3

In [20]:
model.intercept_, model.coef_, answer

(321.0917220883945, array([     -11.02]), 32)

In [18]:

print(len(y))

for i in range(len(X)):
    print( y[i])

24
306.47104267777075
287.8632487831679
274.22709567072326
263.7650469641495
244.8999795835026
230.92477130009246
219.69296757065302
204.0006127441778
191.47911113225902
181.1884378209603
168.84090144274876
158.03876106828983
145.9469081549863
135.15176654413364
124.17930584441193
115.59628021696892
104.52870419171951
94.50132274206535
85.44003745317531
81.56745674593515
75.31600095597217
65.3490627323759
60.4648658313239
51.92783453986888
